<a href="https://colab.research.google.com/github/wadeaT/Cloud-Computing-Course/blob/main/HW3/TurtleSearch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install requests beautifulsoup4

In [ ]:
from bs4 import BeautifulSoup
from nltk.stem import PorterStemmer
from collections import defaultdict
import requests
import json
import nltk
import re
nltk.download('punkt')

class IndexCreator:
    #Class Initialization
    def __init__(self, start_url, firebase_url, max_pages=50):
        self.start_url = start_url # starting URL to proccess
        self.firebase_url = firebase_url.rstrip('/') + '/'  # the firebase URL
        self.max_pages = max_pages  # maximum number of pages to process
        self.index = defaultdict(dict) #for the index to automatically handle new terms
        self.processed_pages = 0 # number of proccessed pages, start at 0
        self.visited_urls = set() #visited URLS

        # Patterns for URLs we want to follow
        self.relevant_patterns = [
            # Core Compute Products
            r'/products/compute',
            r'/compute',  # General compute pages
            r'/kubernetes-engine',  # Google Kubernetes Engine (GKE)
            r'/container-optimized-os',
            r'/run',  # Cloud Run
            r'/functions',  # Cloud Functions
            r'/app-engine',  # App Engine

            # Compute Documentation
            r'/compute/docs',  # All compute docs
            r'/compute/docs/concepts',
            r'/compute/docs/instances',
            r'/compute/docs/disks',
            r'/compute/docs/networking',
            r'/compute/docs/security',
            r'/compute/docs/regions-zones',
            r'/compute/docs/quickstarts',
            r'/compute/docs/tutorials',
            r'/compute/docs/containers',

            # Container and Kubernetes related
            r'/kubernetes-engine/docs',
            r'/container-registry/docs',
            r'/containers',

            # Serverless Computing
            r'/serverless',
            r'/run/docs',  # Cloud Run documentation
            r'/functions/docs',  # Cloud Functions documentation
            r'/app-engine/docs',  # App Engine documentation

            # Infrastructure and VMs
            r'/vpc/docs',  # Virtual Private Cloud
            r'/virtual-machines',
            r'/vmware-engine',

            # Pricing and Planning
            r'/compute/pricing',
            r'/compute/docs/machine-types',
            r'/compute/docs/cpu-platforms',

            # Solutions and Reference
            r'/solutions/compute',
            r'/architecture/compute',
            r'/docs/cloud-computing'
        ]

    #checks if that pattern appears anywhere in the URL string
    #Returns True/False for each check
    def is_relevant_url(self, url):
        """Check if URL matches our relevant patterns"""
        return any(pattern in url for pattern in self.relevant_patterns)

    def extract_content(self, url):
        """Extract meaningful content from a page"""
        try:
            #makes an HTTP request to get the webpage content
            response = requests.get(url)
            #Uses BeautifulSoup to parse the HTML into a soup object for easy navigation
            soup = BeautifulSoup(response.text, 'html.parser')

            # Remove navigation, footer, etc.
            for elem in soup.find_all(['nav', 'footer', 'script', 'style']):
                elem.decompose()

            #extracting main content
            content_sections = []
            # Get main content
            main_content = soup.find('main')
            if main_content:
                content_sections.append(main_content.get_text())

            # Get article content
            article = soup.find('article')
            if article:
                content_sections.append(article.get_text())

            # Get section content
            sections = soup.find_all('section')
            for section in sections:
                content_sections.append(section.get_text())

            return ' '.join(content_sections)
        except Exception as e:
            print(f"Error extracting content from {url}: {e}")
            return ""

    def get_links(self, url):
        """Get relevant links from a page"""
        try:
            response = requests.get(url)
            soup = BeautifulSoup(response.text, 'html.parser')
            links = set() #Using a set to avoid duplicate links

            #finds all <a> tags that have an href attribute
            for link in soup.find_all('a', href=True):
                href = link['href'] #Gets the href value (the URL)
                full_url = ''

                if href.startswith('http'):
                    if 'cloud.google.com' in href:
                        full_url = href
                elif href.startswith('/'):
                    full_url = f"https://cloud.google.com{href}"

                if full_url and self.is_relevant_url(full_url):
                    links.add(full_url)

            return list(links)
        except Exception as e:
            print(f"Error getting links from {url}: {e}")
            return []

    def process_text(self, text):
        """Process text into stemmed words"""
        stop_words = {
            # Articles
            'a', 'an', 'the',

            # Pronouns
            'i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves',
            'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves',
            'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself',
            'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves',
            'this', 'that', "that's", 'these', 'those',
            'who', 'whom', 'whose', 'which', 'what', "what's", 'whatever', 'whoever', 'whomever',

            # Prepositions
            'in', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through',
            'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'on',
            'off', 'over', 'under', 'again', 'further', 'then',

            # Conjunctions
            'and', 'but', 'or', 'as', 'if', 'when', 'than', 'because', 'while', 'where',
            'of', 'until', 'unless', 'since', 'before',

            # Verbs (common forms)
            'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being',
            'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing',
            'would', 'should', 'could', 'might', 'must', 'will', 'shall',
            'can', "can't", 'cannot', 'may', "don't", "doesn't", "didn't",
            "haven't", "hasn't", "hadn't", "isn't", "aren't", "wasn't", "weren't",

            # Adverbs
            'very', 'too', 'so', 'just', 'now', 'here', 'there', 'when',
            'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few',
            'more', 'most', 'other', 'some', 'such',

            # Numbers and quantities
            'one', 'two', 'three', 'four', 'five', 'first', 'second', 'third',
            'many', 'much', 'no', 'none', 'only', 'own', 'same', 'several',

            # Common web/technical terms
            'http', 'https', 'www', 'com', 'net', 'org', 'html', 'php',
            'page', 'website', 'site', 'web', 'click', 'link',

            # Special characters and artifacts
            '-', '&', '_', '/', '\\', '*', '+', '=', '<', '>', '@',
            'ha', 'thi', 'skip', 'nbsp', 'amp',

            # Common filler words
            'well', 'still', 'rather', 'quite', 'even', 'also', 'perhaps',
            'anyway', 'ever', 'often', 'yet', 'usually',

            # Time-related
            'today', 'tomorrow', 'yesterday', 'now', 'then', 'always',
            'never', 'sometimes', 'often', 'usually', 'weekly', 'daily',
            'monthly', 'yearly', 'ago',

            # Misc common words
            'let', 'yes', 'no', 'not', 'ok', 'okay', 'right', 'sure',
            'please', 'thank', 'thanks', 'welcome', 'hi', 'hello', 'bye',
            'good', 'bad', 'nice', 'great', 'best', 'better', 'worse', 'worst'
        }

        words = []
        for word in text.split():
            word = word.lower()
            word = ''.join(c for c in word if c.isalnum())
            if word and word not in stop_words and len(word) > 2:
                words.append(word)
        #Uses Porter Stemming algorithm to reduce words to their root form
        stemmer = PorterStemmer()
        return [stemmer.stem(word) for word in words]

    def create_index(self):
        """Create inverted index from relevant pages"""
        queue = [self.start_url]
        doc_id = 1

        while queue and self.processed_pages < self.max_pages:
            url = queue.pop(0)

            if url in self.visited_urls:
                continue

            print(f"Processing page {self.processed_pages + 1}/{self.max_pages}: {url}")

            # Extract and process content
            content = self.extract_content(url)
            if content:
                words = self.process_text(content)

                # Count word occurrences
                word_counts = defaultdict(int)
                for word in words:
                    word_counts[word] += 1

                # Update index
                for word, count in word_counts.items():
                    self.index[word][str(doc_id)] = {
                        "url": url,
                        "counter": count
                    }

                # Mark as processed
                self.visited_urls.add(url)
                self.processed_pages += 1
                doc_id += 1

                # Get new links to process
                if self.processed_pages < self.max_pages:
                    new_links = self.get_links(url)
                    queue.extend([link for link in new_links if link not in self.visited_urls])

        print(f"\nIndexing complete!")
        print(f"Pages processed: {self.processed_pages}")
        print(f"Unique terms indexed: {len(self.index)}")

    def create_final_data(self):
        """Format index for database storage"""
        return [{
            'term': word,
            'DocId': docs
        } for word, docs in self.index.items()]

    def upload_to_firebase(self):
        """Upload index to Firebase using REST API"""
        print("\nUploading to Firebase...")

        data = self.create_final_data()
        index_url = f"{self.firebase_url}index.json"

        try:
            response = requests.put(index_url, json=data)
            response.raise_for_status()
            print("Upload complete! Data stored in Firebase.")

        except requests.exceptions.RequestException as e:
            print(f"Error uploading to Firebase: {e}")

    def search_word(self, term):
        """Search for a word in the index and return matching document IDs"""
        # Stem the search term to match the index
        stemmer = PorterStemmer()
        stemmed_term = stemmer.stem(term.lower())

        # Get matching documents from the index
        if stemmed_term in self.index:
            return list(self.index[stemmed_term].keys())
        return []

    def get_term_frequency(self, term, doc_id):
        """Get the frequency of a term in a specific document"""
        # Stem the term to match the index
        stemmer = PorterStemmer()
        stemmed_term = stemmer.stem(term.lower())

        # Return the term frequency if it exists, otherwise return 0
        if stemmed_term in self.index and doc_id in self.index[stemmed_term]:
            return self.index[stemmed_term][doc_id]["counter"]
        return 0

    def get_document_url(self, doc_id):
        """Get the URL for a specific document ID"""
        # Search through the index to find the document URL
        for term_data in self.index.values():
            if doc_id in term_data:
                return term_data[doc_id]["url"]
        return None



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
def create_and_upload_index(firebase_url): # 3. RUN THIS
    """Creates an index and uploads it to Firebase with the given Firebase URL"""
    # Static starting URL
    url = "https://cloud.google.com/products/compute"

    # Create the IndexCreator object
    indexer = IndexCreator(url, firebase_url, max_pages=50)

    # Create the index by processing the pages
    indexer.create_index()

    # Upload the generated index to Firebase
    indexer.upload_to_firebase()


In [ ]:
from collections import defaultdict  #4. RUN THIS
import requests
from datetime import datetime
from nltk.stem import PorterStemmer
import nltk
nltk.download('punkt')

# First, get the index from Firebase
def get_index_from_db(firebase_url):
    # Ensure the URL is properly formatted
    firebase_url = firebase_url.rstrip('/')  # Remove trailing slash if present
    index_url = f"{firebase_url}/index.json"  # Add /index.json



    try:
        response = requests.get(index_url)
        response.raise_for_status()
        index_data = response.json()
        index = defaultdict(dict)

        for entry in index_data:
            term = entry['term']
            docs = entry['DocId']
            index[term] = docs

        print("Successfully retrieved index from database")
        return index
    except requests.exceptions.RequestException as e:
        print(f"Error retrieving index from Firebase: {e}")
        return None




[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
import requests   # 5. RUN THIS
from nltk.stem import PorterStemmer
from collections import defaultdict
from datetime import datetime
import nltk
nltk.download('punkt')

class DynamicQueryService:
    def __init__(self, firebase_url):
        self.firebase_url = firebase_url.rstrip('/') + '/'
        self.stemmer = PorterStemmer()

    def get_current_index(self):
        """Fetch the latest index from Firebase"""
        try:
            response = requests.get(f"{self.firebase_url}index.json")
            response.raise_for_status()
            return response.json()
        except Exception as e:
            print(f"Error fetching index: {e}")
            return None

    def normalize_doc_data(self, doc_data):
        """Convert document data to dictionary format if it's a list"""
        if isinstance(doc_data, list):
            # Convert list to dictionary
            return {str(i): item for i, item in enumerate(doc_data) if item is not None}
        return doc_data if isinstance(doc_data, dict) else {}

    def search_word(self, term, current_index):
        """Search for a word in the current index"""
        stemmed_term = self.stemmer.stem(term.lower())

        for entry in current_index:
            if entry['term'] == stemmed_term:
                doc_data = entry.get('DocId', {})
                return self.normalize_doc_data(doc_data)
        return {}

    def get_term_frequency(self, term, doc_id, current_index):
        """Get term frequency from current index"""
        stemmed_term = self.stemmer.stem(term.lower())

        for entry in current_index:
            if entry['term'] == stemmed_term:
                doc_data = self.normalize_doc_data(entry.get('DocId', {}))
                if doc_id in doc_data:
                    return doc_data[doc_id].get('counter', 0)
        return 0

    def get_document_url(self, doc_id, doc_data):
        """Get document URL from document data"""
        normalized_data = self.normalize_doc_data(doc_data)
        if doc_id in normalized_data:
            return normalized_data[doc_id].get('url')
        return None

    def create_query(self, query_terms):
        """Create a query with the latest index data"""
        try:
            # Get the latest index for each query
            current_index = self.get_current_index()
            if not current_index:
                return {"error": "Failed to retrieve index"}

            query_id = str(datetime.now().timestamp())
            doc_sets = []
            term_data = {}

            # Process each search term
            for term in query_terms:
                # Get matching documents for this term
                matching_docs = self.search_word(term, current_index)

                if matching_docs:  # if we found any matches
                    doc_sets.append(set(matching_docs.keys()))
                    stemmed_term = self.stemmer.stem(term.lower())
                    term_data[stemmed_term] = matching_docs

            # If no matching documents were found
            if not doc_sets:
                return {
                    "id": query_id,
                    "message": "No results found for your query.",
                    "results": []
                }

            # Find documents containing all search terms
            results = set.intersection(*doc_sets) if doc_sets else set()

            if not results:
                return {
                    "id": query_id,
                    "message": "No results found for your query.",
                    "results": []
                }

            # Rank results
            ranked_results = []
            for doc_id in results:
                # Calculate score based on term frequency
                score = sum(self.get_term_frequency(term, doc_id, current_index) for term in query_terms)

                # Get URL from the first term data that contains this doc_id
                url = None
                for doc_data in term_data.values():
                    url = self.get_document_url(doc_id, doc_data)
                    if url:
                        break

                if url:
                    ranked_results.append({
                        "doc_id": doc_id,
                        "url": url,
                        "score": score
                    })

            # Sort results by score
            ranked_results.sort(key=lambda x: x["score"], reverse=True)

            return {
                'id': query_id,
                'terms': query_terms,
                'results': ranked_results,
                'timestamp': str(datetime.now())
            }

        except Exception as e:
            import traceback
            print(f"Error details: {traceback.format_exc()}")
            return {'error': f"An error occurred while processing the query: {str(e)}"}

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
#test code
firebase_url = "https://turtlecloud-91905-default-rtdb.firebaseio.com/"
create_and_upload_index(firebase_url)

# Initialize DynamicQueryService with the firebase_url
query_service = DynamicQueryService(firebase_url)

# Test with a simple query
#test_query = ["google", "cloud"]  # input from the search page


# Get results
#results = query_service.create_query(test_query)


Processing page 1/50: https://cloud.google.com/products/compute
Processing page 2/50: https://cloud.google.com/vmware-engine
Processing page 3/50: https://console.cloud.google.com/freetrial/?redirectPath=/compute
Processing page 4/50: https://cloud.google.com/compute/docs/general-purpose-machines#n4_machine_types
Processing page 5/50: https://cloud.google.com/containers
Processing page 6/50: https://console.cloud.google.com/compute/instances?_gl=1*qsgqme*_ga*MjA3OTc3Nzk3NS4xNzA0ODkxMjM2*_ga_4LYFWVHBEB*MTcwNTU1ODg4OC4yNy4xLjE3MDU1NjM3MjYuMC4wLjA.
Processing page 7/50: https://cloud.google.com/compute/docs/networking/network-overview
Processing page 8/50: https://cloud.google.com/kubernetes-engine
Processing page 9/50: https://cloud.google.com/compute/docs/regions-zones#available
Processing page 10/50: https://cloud.google.com/compute/docs/images/creating-custom-windows-byol-images?hl=en
Processing page 11/50: https://cloud.google.com/compute/docs/resource-quotas
Processing page 12/50: h

In [ ]:
import requests

def setup_admin_credentials(firebase_url, username, password):
    # Ensure URL is properly formatted
    firebase_url = firebase_url.rstrip('/') + '/'
    admin_url = f"{firebase_url}admin_credentials.json"

    # Create admin credentials object
    credentials = {
        "username": username,
        "password": password
    }

    try:
        # Upload to Firebase using PUT request
        response = requests.put(admin_url, json=credentials)
        response.raise_for_status()
        print("Admin credentials successfully added to Firebase")
    except Exception as e:
        print(f"Error setting up admin credentials: {e}")

#existing Firebase URL
firebase_url = "https://turtlecloud-91905-default-rtdb.firebaseio.com/"

# Call the function with your desired username and password
setup_admin_credentials(firebase_url, "admin", "admin123")

Admin credentials successfully added to Firebase


In [ ]:
# Admin functionality for the search engine
from google.colab import output

class AdminService:
    def __init__(self, firebase_url):
        self.firebase_url = firebase_url.rstrip('/') + '/'
        self.admin_credentials_url = f"{self.firebase_url}admin_credentials.json"
        self.index_url = f"{self.firebase_url}index.json"
        self.stemmer = PorterStemmer()

    def verify_credentials(self, username, password):
        """Verify admin credentials"""
        try:
            response = requests.get(self.admin_credentials_url)
            credentials = response.json()
            return (credentials.get('username') == username and
                    credentials.get('password') == password)
        except Exception as e:
            print(f"Error verifying credentials: {e}")
            return False

    def get_current_index(self):
        """Retrieve current index from Firebase"""
        try:
            response = requests.get(self.index_url)
            return response.json()
        except Exception as e:
            print(f"Error getting index: {e}")
            return None

    def add_term(self, term, url, count):
        """Add new term or URL to existing term"""
        try:
            # Get current index
            current_index = self.get_current_index()
            if not current_index:
                return False, "Failed to retrieve index"

            # Stem the term
            stemmed_term = self.stemmer.stem(term.lower())

            # Find if term exists
            term_entry = None
            for entry in current_index:
                if entry['term'] == stemmed_term:
                    term_entry = entry
                    break

            if term_entry:
                # Term exists, add new URL if not present
                doc_id = str(len(term_entry['DocId']) + 1)
                term_entry['DocId'][doc_id] = {
                    "url": url,
                    "counter": count
                }
            else:
                # Create new term entry
                new_entry = {
                    'term': stemmed_term,
                    'DocId': {
                        "1": {
                            "url": url,
                            "counter": count
                        }
                    }
                }
                current_index.append(new_entry)

            # Update Firebase
            response = requests.put(self.index_url, json=current_index)
            response.raise_for_status()
            return True, "Term added successfully"

        except Exception as e:
            return False, f"Error adding term: {str(e)}"

    def remove_term(self, term):
        """Remove a term from the index"""
        try:
            # Get current index
            current_index = self.get_current_index()
            if not current_index:
                return False, "Failed to retrieve index"

            # Stem the term
            stemmed_term = self.stemmer.stem(term.lower())

            # Remove term if it exists
            new_index = [entry for entry in current_index if entry['term'] != stemmed_term]

            if len(new_index) == len(current_index):
                return False, "Term not found in index"

            # Update Firebase
            response = requests.put(self.index_url, json=new_index)
            response.raise_for_status()
            return True, "Term removed successfully"

        except Exception as e:
            return False, f"Error removing term: {str(e)}"

    def remove_url_from_term(self, term, url):
        """Remove specific URL from a term"""
        try:
            # Get current index
            current_index = self.get_current_index()
            if not current_index:
                return False, "Failed to retrieve index"

            # Stem the term
            stemmed_term = self.stemmer.stem(term.lower())

            # Find term and remove URL
            for entry in current_index:
                if entry['term'] == stemmed_term:
                    # Find and remove the URL
                    docs_to_remove = []
                    for doc_id, doc_data in entry['DocId'].items():
                        if doc_data['url'] == url:
                            docs_to_remove.append(doc_id)

                    for doc_id in docs_to_remove:
                        del entry['DocId'][doc_id]

                    # Remove term if no URLs left
                    if not entry['DocId']:
                        current_index.remove(entry)

                    # Update Firebase
                    response = requests.put(self.index_url, json=current_index)
                    response.raise_for_status()
                    return True, "URL removed successfully"

            return False, "Term or URL not found"

        except Exception as e:
            return False, f"Error removing URL: {str(e)}"

def display_admin_page():
    """Generate HTML for admin page"""
    admin_page_html = """
    <div id="adminPage" style="display: block;">
        <div style="text-align: center; padding: 50px; background-color: #f0f4f8; min-height: 100vh; font-family: Arial, sans-serif;">
            <!-- Login Section -->
            <div id="loginSection" style="display: block;">
                <h2>Admin Login</h2>
                <div style="background-color: white; padding: 30px; border-radius: 15px; max-width: 400px; margin: 0 auto;">
                    <input type="text" id="username" placeholder="Username" style="width: 80%; padding: 10px; margin: 10px;">
                    <input type="password" id="password" placeholder="Password" style="width: 80%; padding: 10px; margin: 10px;">
                    <button onclick="adminLogin()" style="padding: 10px 20px; background-color: #4CAF50; color: white; border: none; border-radius: 5px; cursor: pointer;">
                        Login
                    </button>
                </div>
            </div>

            <!-- Admin Dashboard -->
            <div id="adminDashboard" style="display: none;">
                <h2>Admin Dashboard</h2>
                <div style="background-color: white; padding: 30px; border-radius: 15px; max-width: 800px; margin: 0 auto;">
                    <!-- Add Term Section -->
                    <div style="margin-bottom: 30px;">
                        <h3>Add Term</h3>
                        <input type="text" id="newTerm" placeholder="Term" style="width: 30%; padding: 10px; margin: 5px;">
                        <input type="text" id="newUrl" placeholder="URL" style="width: 30%; padding: 10px; margin: 5px;">
                        <input type="number" id="termCount" placeholder="Count" style="width: 20%; padding: 10px; margin: 5px;">
                        <button onclick="addTerm()" style="padding: 10px 20px; background-color: #4CAF50; color: white; border: none; border-radius: 5px; cursor: pointer;">
                            Add
                        </button>
                    </div>

                    <!-- Remove Term Section -->
                    <div style="margin-bottom: 30px;">
                        <h3>Remove Term</h3>
                        <input type="text" id="termToRemove" placeholder="Term to remove" style="width: 60%; padding: 10px; margin: 5px;">
                        <button onclick="removeTerm()" style="padding: 10px 20px; background-color: #ff4444; color: white; border: none; border-radius: 5px; cursor: pointer;">
                            Remove
                        </button>
                    </div>

                    <!-- Remove URL from Term Section -->
                    <div>
                        <h3>Remove URL from Term</h3>
                        <input type="text" id="termForUrl" placeholder="Term" style="width: 30%; padding: 10px; margin: 5px;">
                        <input type="text" id="urlToRemove" placeholder="URL to remove" style="width: 50%; padding: 10px; margin: 5px;">
                        <button onclick="removeUrl()" style="padding: 10px 20px; background-color: #ff4444; color: white; border: none; border-radius: 5px; cursor: pointer;">
                            Remove URL
                        </button>
                    </div>
                </div>

                <!-- Logout Button -->
                <button onclick="adminLogout()" style="margin-top: 20px; padding: 10px 20px; background-color: #666; color: white; border: none; border-radius: 5px; cursor: pointer;">
                    Logout
                </button>
            </div>
        </div>
    </div>

    <script>
        function adminLogin() {
            const username = document.getElementById('username').value;
            const password = document.getElementById('password').value;

            google.colab.kernel.invokeFunction(
                'notebook.handleAdminLogin',
                [username, password],
                {}
            );
        }

        function adminLogout() {
            document.getElementById('loginSection').style.display = 'block';
            document.getElementById('adminDashboard').style.display = 'none';
            document.getElementById('username').value = '';
            document.getElementById('password').value = '';
        }

        function addTerm() {
            const term = document.getElementById('newTerm').value;
            const url = document.getElementById('newUrl').value;
            const count = document.getElementById('termCount').value;

            google.colab.kernel.invokeFunction(
                'notebook.handleAddTerm',
                [term, url, parseInt(count)],
                {}
            );
        }

        function removeTerm() {
            const term = document.getElementById('termToRemove').value;

            google.colab.kernel.invokeFunction(
                'notebook.handleRemoveTerm',
                [term],
                {}
            );
        }

        function removeUrl() {
            const term = document.getElementById('termForUrl').value;
            const url = document.getElementById('urlToRemove').value;

            google.colab.kernel.invokeFunction(
                'notebook.handleRemoveUrl',
                [term, url],
                {}
            );
        }

        window.showAdminDashboard = function() {
            document.getElementById('loginSection').style.display = 'none';
            document.getElementById('adminDashboard').style.display = 'block';
        }

        window.showMessage = function(message, isError) {
            alert(message);
        }
    </script>
    """
    display(HTML(admin_page_html))

# Register callback handlers

def handle_admin_login(username, password):
    try:
        admin_service = AdminService(firebase_url)

        success = admin_service.verify_credentials(username, password)
        if success:
            display(Javascript('window.showAdminDashboard()'))
            display(Javascript('window.showMessage("Login successful!", false)'))
        else:
            display(Javascript('window.showMessage("Invalid credentials", true)'))

    except Exception as e:
        print(f"Login error: {str(e)}")  # Debug print
        display(Javascript(f'window.showMessage("Error during login: {str(e)}", true)'))

def handle_add_term(term, url, count):
    admin_service = AdminService(firebase_url)
    success, message = admin_service.add_term(term, url, count)
    display(Javascript(f'window.showMessage("{message}", {str(not success).lower()})'))

def handle_remove_term(term):
    admin_service = AdminService(firebase_url)
    success, message = admin_service.remove_term(term)
    display(Javascript(f'window.showMessage("{message}", {str(not success).lower()})'))

def handle_remove_url(term, url):
    admin_service = AdminService(firebase_url)
    success, message = admin_service.remove_url_from_term(term, url)
    display(Javascript(f'window.showMessage("{message}", {str(not success).lower()})'))

# Register the callbacks
output.register_callback('notebook.handleAdminLogin', handle_admin_login)
output.register_callback('notebook.handleAddTerm', handle_add_term)
output.register_callback('notebook.handleRemoveTerm', handle_remove_term)
output.register_callback('notebook.handleRemoveUrl', handle_remove_url)

In [ ]:
# Make sure these imports are at the top of your code
from google.colab import output
from IPython.display import display, HTML, Javascript
import requests
import json

def handle_statistics(*args):
    """Handle fetching and displaying statistics"""
    try:
        # Fetch index data from Firebase
        firebase_url = "https://turtlecloud-91905-default-rtdb.firebaseio.com/"

        response = requests.get(f"{firebase_url}index.json")
        index_data = response.json()

        # Process data - only using terms from inverted index
        word_frequencies = {}
        url_frequencies = {}

        # Calculate frequencies directly from index_data
        for entry in index_data:
            term = entry['term']
            docs = entry['DocId']

            # Handle both array and object DocId formats
            if isinstance(docs, list):
                docs = {str(i): doc for i, doc in enumerate(docs) if doc is not None}

            term_total = 0
            for doc_id, doc_data in docs.items():
                if isinstance(doc_data, dict) and 'counter' in doc_data and 'url' in doc_data:
                    count = doc_data['counter']
                    term_total += count

                    # Track URL frequencies
                    url = doc_data['url']
                    if url not in url_frequencies:
                        url_frequencies[url] = 0
                    url_frequencies[url] += count

            word_frequencies[term] = term_total

        # Prepare statistics data
        stats = {
            'summary': {
                'totalWords': sum(word_frequencies.values()),
                'uniqueWords': len(word_frequencies),
                'uniqueUrls': len(url_frequencies)
            },
            'topWords': sorted(
                [{'term': term, 'count': count}
                 for term, count in word_frequencies.items()],
                key=lambda x: x['count'],
                reverse=True
            )[:10]
        }


        # Update the statistics content
        js_code = f"""
        (function() {{

            const stats = {json.dumps(stats)};

            document.getElementById('statisticsPage').style.display = 'block';
            document.getElementById('searchPage').style.display = 'none';
            document.getElementById('resultsPage').style.display = 'none';
            document.getElementById('adminPage').style.display = 'none';

            document.getElementById('statisticsContent').innerHTML = `
                <div style="padding: 20px; background-color: #f0f4f8;">
                    <h1 style="text-align: center; color: #2d3748; margin-bottom: 30px;">Search Engine Statistics</h1>

                    <!-- Summary Cards -->
                    <div style="display: grid; grid-template-columns: repeat(3, 1fr); gap: 20px; margin-bottom: 30px;">
                        <div style="background: white; padding: 20px; border-radius: 10px; text-align: center; box-shadow: 0 2px 4px rgba(0,0,0,0.1);">
                            <h3 style="color: #4a5568; margin-bottom: 10px;">Total Words</h3>
                            <p style="font-size: 24px; color: #2d3748;">${{stats.summary.totalWords.toLocaleString()}}</p>
                        </div>
                        <div style="background: white; padding: 20px; border-radius: 10px; text-align: center; box-shadow: 0 2px 4px rgba(0,0,0,0.1);">
                            <h3 style="color: #4a5568; margin-bottom: 10px;">Unique Terms</h3>
                            <p style="font-size: 24px; color: #2d3748;">${{stats.summary.uniqueWords.toLocaleString()}}</p>
                        </div>
                        <div style="background: white; padding: 20px; border-radius: 10px; text-align: center; box-shadow: 0 2px 4px rgba(0,0,0,0.1);">
                            <h3 style="color: #4a5568; margin-bottom: 10px;">Unique URLs</h3>
                            <p style="font-size: 24px; color: #2d3748;">${{stats.summary.uniqueUrls.toLocaleString()}}</p>
                        </div>
                    </div>

                    <!-- Most Frequent Terms Chart -->
                    <div style="background: white; padding: 20px; border-radius: 10px; box-shadow: 0 2px 4px rgba(0,0,0,0.1); margin-bottom: 20px;">
                        <h3 style="color: #4a5568; margin-bottom: 20px;">Most Frequent Terms</h3>
                        <div style="height: 400px;">
                            <canvas id="topWordsChart"></canvas>
                        </div>
                    </div>

                    <!-- Back to Search Button -->
                    <div style="text-align: center; margin-top: 30px;">
                        <button onclick="showSearchPage()"
                                style="padding: 10px 20px; background-color: #63b3ed; color: white;
                                       border: none; border-radius: 20px; cursor: pointer;">
                            Back to Search
                        </button>
                    </div>
                </div>
            `;

            // Load Chart.js if needed
            if (!window.Chart) {{

                const script = document.createElement('script');
                script.src = 'https://cdn.jsdelivr.net/npm/chart.js';
                script.onload = () => createChart(stats);
                document.head.appendChild(script);
            }} else {{

                createChart(stats);
            }}

            function createChart(stats) {{

                const ctx = document.getElementById('topWordsChart').getContext('2d');
                new Chart(ctx, {{
                    type: 'bar',
                    data: {{
                        labels: stats.topWords.map(item => item.term),
                        datasets: [{{
                            label: 'Frequency',
                            data: stats.topWords.map(item => item.count),
                            backgroundColor: '#4299e1',
                            borderColor: '#3182ce',
                            borderWidth: 1
                        }}]
                    }},
                    options: {{
                        responsive: true,
                        maintainAspectRatio: false,
                        scales: {{
                            y: {{
                                beginAtZero: true,
                                title: {{
                                    display: true,
                                    text: 'Frequency'
                                }}
                            }},
                            x: {{
                                title: {{
                                    display: true,
                                    text: 'Terms'
                                }},
                                ticks: {{
                                    maxRotation: 45,
                                    minRotation: 45
                                }}
                            }}
                        }},
                        plugins: {{
                            legend: {{
                                display: false
                            }},
                            title: {{
                                display: true,
                                text: 'Most Frequent Terms in the Index'
                            }}
                        }}
                    }}
                }});

            }}
        }})();
        """

        display(Javascript(js_code))

    except Exception as e:
        import traceback
        error_msg = str(e)
        print(f"Error loading statistics: {error_msg}")
        print(traceback.format_exc())

        error_js = """
        console.error('Error in statistics handling');
        document.getElementById('statisticsContent').innerHTML = `
            <div style="color: red; padding: 20px;">
                Error loading statistics. Please try again.
                <br><br>
                <button onclick="showSearchPage()"
                        style="padding: 10px 20px; background-color: #63b3ed; color: white;
                               border: none; border-radius: 20px; cursor: pointer;">
                    Back to Search
                </button>
            </div>
        `;
        """
        display(Javascript(error_js))

# Register the callback
output.register_callback('notebook.handleStatistics', handle_statistics)

# Make sure the statistics div exists in your HTML
statistics_div = """
<div id="statisticsPage" style="display: none;">
    <div id="statisticsContent">
        <!-- Content will be populated by handle_statistics() -->
    </div>
</div>
"""
display(HTML(statistics_div))

In [ ]:
from google.colab import output
from IPython.display import display, HTML, Javascript

def display_search_page(previous_search=""):
    search_page_html = """
    <!-- Statistics Page Container -->
    <div id="statisticsPage" style="display: none;">
        <div id="statisticsContent">
            <!-- Content will be populated by handle_statistics() -->
        </div>
    </div>

    <!-- Admin Page -->
    <div id="adminPage" style="display: none;">
        <div style="text-align: center; padding: 50px; background: linear-gradient(135deg, #e0f4e8 0%, #e6f3ff 100%); min-height: 100vh; font-family: Arial, sans-serif;">
            <!-- Back to Search Button -->
            <button onclick="showSearchPage()"
                    style="position: absolute;
                           top: 20px;
                           left: 20px;
                           padding: 8px 16px;
                           background-color: #4FB3A3;
                           color: white;
                           border: none;
                           border-radius: 20px;
                           cursor: pointer;">
                ← Back to Search
            </button>

            <!-- Login Section -->
            <div id="loginSection" style="display: block;">
                <h2 style="color: black;">Admin Login</h2>
                <div style="background-color: rgba(255, 255, 255, 0.9); padding: 30px; border-radius: 15px; max-width: 400px; margin: 0 auto;">
                    <input type="text" id="username" placeholder="Username" style="width: 80%; padding: 10px; margin: 10px;">
                    <input type="password" id="password" placeholder="Password" style="width: 80%; padding: 10px; margin: 10px;">
                    <button onclick="adminLogin()" style="padding: 10px 20px; background-color: #4FB3A3; color: white; border: none; border-radius: 5px; cursor: pointer;">
                        Login
                    </button>
                </div>
            </div>

            <!-- Admin Dashboard -->
            <div id="adminDashboard" style="display: none;">
                <h2 style="text-align: center; margin-bottom: 30px; color: black;">Admin Dashboard</h2>

                <!-- Add Term Section -->
                <div style="background-color: white; padding: 20px; border-radius: 10px; margin-bottom: 20px; box-shadow: 0 2px 4px rgba(0,0,0,0.1);">
                    <h3 style="margin-bottom: 15px; color: black;">Add New Term</h3>
                    <div style="display: flex; gap: 10px; margin-bottom: 10px;">
                        <input type="text" id="newTerm" placeholder="Enter term"
                               style="flex: 1; padding: 8px; border: 1px solid #ddd; border-radius: 4px;">
                        <input type="text" id="newUrl" placeholder="Enter URL"
                               style="flex: 2; padding: 8px; border: 1px solid #ddd; border-radius: 4px;">
                        <input type="number" id="termCount" placeholder="Count"
                               style="width: 100px; padding: 8px; border: 1px solid #ddd; border-radius: 4px;">
                    </div>
                    <button onclick="addTerm()"
                            style="background-color: #4CAF50; color: white; padding: 8px 16px;
                                   border: none; border-radius: 4px; cursor: pointer;">
                        Add Term
                    </button>
                </div>

                <!-- Remove Term Section -->
                <div style="background-color: white; padding: 20px; border-radius: 10px; margin-bottom: 20px; box-shadow: 0 2px 4px rgba(0,0,0,0.1);">
                    <h3 style="margin-bottom: 15px; color: black;">Remove Term</h3>
                    <div style="display: flex; gap: 10px;">
                        <input type="text" id="termToRemove" placeholder="Enter term to remove"
                               style="flex: 1; padding: 8px; border: 1px solid #ddd; border-radius: 4px;">
                        <button onclick="removeTerm()"
                                style="background-color: #f44336; color: white; padding: 8px 16px;
                                       border: none; border-radius: 4px; cursor: pointer;">
                            Remove Term
                        </button>
                    </div>
                </div>

                <!-- Remove URL from Term Section -->
                <div style="background-color: white; padding: 20px; border-radius: 10px; margin-bottom: 20px; box-shadow: 0 2px 4px rgba(0,0,0,0.1);">
                    <h3 style="margin-bottom: 15px; color: black;">Remove URL from Term</h3>
                    <div style="display: flex; flex-direction: column; gap: 10px;">
                        <input type="text" id="termForUrl" placeholder="Enter term"
                               style="padding: 8px; border: 1px solid #ddd; border-radius: 4px;">
                        <input type="text" id="urlToRemove" placeholder="Enter URL to remove"
                               style="padding: 8px; border: 1px solid #ddd; border-radius: 4px;">
                        <button onclick="removeUrl()"
                                style="background-color: #f44336; color: white; padding: 8px 16px;
                                       border: none; border-radius: 4px; cursor: pointer; width: fit-content;">
                            Remove URL
                        </button>
                    </div>
                </div>

                <!-- Logout Button -->
                <button onclick="adminLogout()"
                        style="display: block; margin: 20px auto; background-color: #666;
                               color: white; padding: 8px 16px; border: none; border-radius: 4px;
                               cursor: pointer;">
                    Logout
                </button>
            </div>
        </div>
    </div>

    <!-- Search Page -->
<div id="searchPage" style="display: block;">
    <div style="text-align: center;
                padding: 30px;
                background-color: #f0f4f8;
                height: 86vh;
                font-family: Arial, sans-serif;
                position: relative;">
        <!-- Navigation Buttons Container -->
        <div style="position: absolute;
                    top: 20px;
                    right: 20px;
                    display: flex;
                    gap: 10px;">
            <button onclick="showStatisticsPage()"
                    style="padding: 8px 16px;
                           background-color: #63b3ed;
                           color: white;
                           border: none;
                           border-radius: 20px;
                           cursor: pointer;
                           transition: background-color 0.2s;">
                📊 Statistics
            </button>
            <button onclick="showAdminPage()"
                    style="padding: 8px 16px;
                           background-color: #63b3ed;
                           color: white;
                           border: none;
                           border-radius: 20px;
                           cursor: pointer;
                           transition: background-color 0.2s;">
                Admin Login
            </button>
        </div>

        <!-- Logo and Title Container -->
        <div style="margin-bottom: 20px;
                    padding: 15px;
                    background-color: white;
                    border-radius: 15px;
                    display: inline-block;
                    box-shadow: 0 2px 10px rgba(0,0,0,0.1);">
            <span style="font-size: 48px;">🐢 ☁️</span>
            <h1 style="font-size: 32px;
                       color: black;
                       margin-top: 10px;
                       margin-bottom: 0;">TurtleNimbus</h1>
        </div>

        <!-- Search Container -->
        <div style="background-color: white;
                    padding: 25px;
                    border-radius: 15px;
                    box-shadow: 0 2px 10px rgba(0,0,0,0.1);
                    max-width: 600px;
                    margin: 0 auto;">
            <!-- Search Input -->
            <div style="display: flex;
                        gap: 10px;
                        margin-bottom: 15px;">
                <input type="text" id="search_input"
                       style="flex: 1;
                              padding: 12px 20px;
                              border: 2px solid #e2e8f0;
                              border-radius: 25px;
                              font-size: 16px;
                              outline: none;
                              transition: border-color 0.3s ease;
                              background-color: #f8fafc;
                              color: black;"
                       placeholder="Search Google Cloud documentation...">

                <!-- Search Button -->
                <button onclick="performSearch()"
                        style="padding: 12px 24px;
                               background-color: #63b3ed;
                               color: white;
                               border: none;
                               border-radius: 25px;
                               cursor: pointer;
                               font-size: 16px;
                               transition: background-color 0.3s ease;
                               box-shadow: 0 2px 4px rgba(0,0,0,0.1);">
                    Search
                </button>
            </div>

            <!-- Search Tip -->
            <div style="color: #666;
                        font-size: 14px;">
                Try searching for "compute engine" or "kubernetes"
            </div>
        </div>
    </div>
</div>

<!-- Results Page -->
<div id="resultsPage" style="display: none;">
    <!-- Navigation Bar -->
    <div style="background-color: white;
                padding: 10px 20px;
                box-shadow: 0 2px 4px rgba(0,0,0,0.1);
                position: fixed;
                top: 0;
                left: 0;
                right: 0;
                display: flex;
                align-items: center;
                z-index: 1000;">
        <!-- Logo -->
        <div onclick="showSearchPage()"
             style="cursor: pointer;
                    display: flex;
                    align-items: center;">
            <span style="font-size: 24px;">🐢 ☁️</span>
            <span style="font-size: 20px;
                        color: black;
                        margin-left: 10px;">TurtleNimbus</span>
        </div>

        <!-- Search Input in Nav -->
        <div style="margin-left: 30px;
                    flex-grow: 1;
                    max-width: 600px;">
            <div style="display: flex;
                        gap: 10px;">
                <input type="text"
                       id="nav_search_input"
                       style="flex: 1;
                              padding: 8px 15px;
                              border: 2px solid #e2e8f0;
                              border-radius: 20px;
                              font-size: 14px;
                              outline: none;">
                <button onclick="performSearch(true)"
                        style="padding: 8px 16px;
                               background-color: #63b3ed;
                               color: white;
                               border: none;
                               border-radius: 20px;
                               cursor: pointer;
                               font-size: 14px;
                               transition: background-color 0.2s;">
                    Search
                </button>
            </div>
        </div>

        <!-- Navigation Buttons -->
        <div style="display: flex;
                    gap: 10px;
                    margin-left: auto;">
            <button onclick="showStatisticsPage()"
                    style="padding: 8px 16px;
                           background-color: #63b3ed;
                           color: white;
                           border: none;
                           border-radius: 20px;
                           cursor: pointer;
                           transition: background-color 0.2s;">
                📊 Statistics
            </button>
            <button onclick="showAdminPage()"
                    style="padding: 8px 16px;
                           background-color: #63b3ed;
                           color: white;
                           border: none;
                           border-radius: 20px;
                           cursor: pointer;
                           transition: background-color 0.2s;">
                Admin Login
            </button>
        </div>
    </div>

    <!-- Results Container -->
    <div id="searchResults"
         style="margin-top: 60px;
                padding: 20px;
                background-color: #f0f4f8;">
        <!-- Results will be inserted here -->
    </div>
</div>

    <script>
        (function(){if(!window.chatbase||window.chatbase("getState")!=="initialized"){window.chatbase=(...arguments)=>{if(!window.chatbase.q){window.chatbase.q=[]}window.chatbase.q.push(arguments)};window.chatbase=new Proxy(window.chatbase,{get(target,prop){if(prop==="q"){return target.q}return(...args)=>target(prop,...args)}})}const onLoad=function(){const script=document.createElement("script");script.src="https://www.chatbase.co/embed.min.js";script.id="nqZkwrfOYX29-OGZp4jah";script.domain="www.chatbase.co";document.body.appendChild(script)};if(document.readyState==="complete"){onLoad()}else{window.addEventListener("load",onLoad)}})();
        function showSearchPage() {
            document.getElementById('statisticsPage').style.display = 'none';
            document.getElementById('resultsPage').style.display = 'none';
            document.getElementById('adminPage').style.display = 'none';
            document.getElementById('searchPage').style.display = 'block';

            document.getElementById('search_input').value = '';
            if (document.getElementById('nav_search_input')) {
                document.getElementById('nav_search_input').value = '';
            }
        }

        function showResultsPage() {
            document.getElementById('searchPage').style.display = 'none';
            document.getElementById('statisticsPage').style.display = 'none';
            document.getElementById('adminPage').style.display = 'none';
            document.getElementById('resultsPage').style.display = 'block';
        }

        function showAdminPage() {
            document.getElementById('searchPage').style.display = 'none';
            document.getElementById('resultsPage').style.display = 'none';
            document.getElementById('statisticsPage').style.display = 'none';
            document.getElementById('adminPage').style.display = 'block';
            document.getElementById('adminDashboard').style.display = 'none';
            document.getElementById('loginSection').style.display = 'block';
        }

        window.showAdminDashboard = function() {
            document.getElementById('loginSection').style.display = 'none';
            document.getElementById('adminDashboard').style.display = 'block';
        }

        function adminLogout() {
            document.getElementById('loginSection').style.display = 'block';
            document.getElementById('adminDashboard').style.display = 'none';
            document.getElementById('username').value = '';
            document.getElementById('password').value = '';
        }

        function performSearch(fromNav = false) {
            const searchInput = fromNav ?
                document.getElementById('nav_search_input') :
                document.getElementById('search_input');
            const searchText = searchInput.value;

            if (searchText) {
                const terms = searchText.split(' ').filter(term => term.trim() !== '');

                document.getElementById('search_input').value = searchText;
                document.getElementById('nav_search_input').value = searchText;

                document.getElementById('searchResults').innerHTML =
                    '<div style="text-align: center; padding: 20px;">Loading results...</div>';

                showResultsPage();

                google.colab.kernel.invokeFunction(
                    'notebook.handleSearch',
                    [[terms, 1]],
                    {}
                );
            } else {
                alert('Please enter some text before searching.');
            }
        }

        function performPageChange(page) {
            const searchInput = document.getElementById('nav_search_input').value ||
                               document.getElementById('search_input').value;
            const terms = searchInput.split(' ').filter(term => term.trim() !== '');

            document.getElementById('searchResults').innerHTML =
                '<div style="text-align: center; padding: 20px;">Loading results...</div>';

            google.colab.kernel.invokeFunction(
                'notebook.handleSearch',
                [[terms, page]],
                {}
            );
        }

        function showStatisticsPage() {
            document.getElementById('searchPage').style.display = 'none';
            document.getElementById('resultsPage').style.display = 'none';
            document.getElementById('adminPage').style.display = 'none';

            const statisticsPage = document.getElementById('statisticsPage');
            statisticsPage.style.display = 'block';

            document.getElementById('statisticsContent').innerHTML = `
                <div style="text-align: center; padding: 50px;">
                    <h2>Loading Statistics...</h2>
                </div>
            `;

            google.colab.kernel.invokeFunction(
                'notebook.handleStatistics',
                [],
                {}
            );
        }

        function adminLogin() {
            const username = document.getElementById('username').value;
            const password = document.getElementById('password').value;

            if (!username || !password) {
                window.showMessage("Please enter both username and password", true);
                return;
            }

            google.colab.kernel.invokeFunction(
                'notebook.handleAdminLogin',
                [username, password],
                {}
            );
        }

        function addTerm() {
            const term = document.getElementById('newTerm').value;
            const url = document.getElementById('newUrl').value;
            const count = document.getElementById('termCount').value;

            if (!term || !url || !count) {
                window.showMessage("Please fill in all fields", true);
                return;
            }

            google.colab.kernel.invokeFunction(
                'notebook.handleAddTerm',
                [term, url, parseInt(count)],
                {}
            );

            document.getElementById('newTerm').value = '';
            document.getElementById('newUrl').value = '';
            document.getElementById('termCount').value = '';
        }

        function removeTerm() {
            const term = document.getElementById('termToRemove').value;
            if (!term) {
                window.showMessage("Please enter a term to remove", true);
                return;
            }

            google.colab.kernel.invokeFunction(
                'notebook.handleRemoveTerm',
                [term],
                {}
            );

            document.getElementById('termToRemove').value = '';
        }

        function removeUrl() {
            const term = document.getElementById('termForUrl').value;
            const url = document.getElementById('urlToRemove').value;

            if (!term || !url) {
                window.showMessage("Please fill in both term and URL", true);
                return;
            }

            google.colab.kernel.invokeFunction(
                'notebook.handleRemoveUrl',
                [term, url],
                {}
            );

            document.getElementById('termForUrl').value = '';
            document.getElementById('urlToRemove').value = '';
        }

        window.displayResults = function(resultsHtml) {
            document.getElementById('searchResults').innerHTML = resultsHtml;
        }

        window.showMessage = function(message, isError) {
            alert(message);
        }

        window.updateStatistics = function(stats) {
            document.getElementById('statisticsContent').innerHTML = `
                <div style="text-align: center; padding: 50px; background-color: #f0f4f8; min-height: 100vh; font-family: Arial, sans-serif;">
                    <!-- Back to Search Button -->
                    <button onclick="showSearchPage()"
                            style="position: absolute;
                                   top: 20px;
                                   left: 20px;
                                   padding: 8px 16px;
                                   background-color: #63b3ed;
                                   color: white;
                                   border: none;
                                   border-radius: 20px;
                                   cursor: pointer;">
                        ← Back to Search
                    </button>

                    <!-- Navigation Buttons Container -->
                    <div style="position: absolute;
                                top: 20px;
                                right: 20px;
                                display: flex;
                                gap: 10px;">
                        <button onclick="showAdminPage()"
                                style="padding: 8px 16px;
                                       background-color: #63b3ed;
                                       color: white;
                                       border: none;
                                       border-radius: 20px;
                                       cursor: pointer;">
                            Admin Login
                        </button>
                    </div>

                    <h2 style="color: black; margin-bottom: 30px;">Search Statistics</h2>

                    <!-- Summary Cards -->
                    <div style="display: flex; justify-content: center; gap: 20px; margin-bottom: 40px;">
                        <div style="background: white; padding: 20px; border-radius: 10px; width: 200px;">
                            <h3>Total Words</h3>
                            <div id="totalWords">${stats.totalWords}</div>
                        </div>
                        <div style="background: white; padding: 20px; border-radius: 10px; width: 200px;">
                            <h3>Unique Words</h3>
                            <div id="uniqueWords">${stats.uniqueWords}</div>
                        </div>
                        <div style="background: white; padding: 20px; border-radius: 10px; width: 200px;">
                            <h3>Unique URLs</h3>
                            <div id="uniqueUrls">${stats.uniqueUrls}</div>
                        </div>
                    </div>

                    <!-- Charts Grid -->
                    <div style="display: grid; grid-template-columns: 1fr 1fr; gap: 20px; max-width: 1200px; margin: 0 auto;">
                        <div style="background: white; padding: 20px; border-radius: 10px;">
                            <h3>Top Words</h3>
                            <canvas id="topWordsChart" style="height: 300px;"></canvas>
                        </div>
                        <div style="background: white; padding: 20px; border-radius: 10px;">
                            <h3>Least Used Words</h3>
                            <canvas id="leastWordsChart" style="height: 300px;"></canvas>
                        </div>
                        <div style="background: white; padding: 20px; border-radius: 10px;">
                            <h3>Word Distribution</h3>
                            <canvas id="distributionChart" style="height: 300px;"></canvas>
                        </div>
                        <div style="background: white; padding: 20px; border-radius: 10px;">
                            <h3>Top URLs</h3>
                            <canvas id="topUrlsChart" style="height: 300px;"></canvas>
                        </div>
                    </div>
                </div>
            `;

            // Create charts
            new Chart(document.getElementById('topWordsChart'), {
                type: 'bar',
                data: {
                    labels: stats.topWords.map(item => item.term),
                    datasets: [{
                        label: 'Frequency',
                        data: stats.topWords.map(item => item.count),
                        backgroundColor: '#8884d8'
                    }]
                },
                options: {
                    responsive: true,
                    maintainAspectRatio: false
                }
            });

            new Chart(document.getElementById('leastWordsChart'), {
                type: 'bar',
                data: {
                    labels: stats.leastWords.map(item => item.term),
                    datasets: [{
                        label: 'Frequency',
                        data: stats.leastWords.map(item => item.count),
                        backgroundColor: '#82ca9d'
                    }]
                },
                options: {
                    responsive: true,
                    maintainAspectRatio: false
                }
            });

            new Chart(document.getElementById('distributionChart'), {
                type: 'pie',
                data: {
                    labels: stats.distributions.map(item => item.range),
                    datasets: [{
                        data: stats.distributions.map(item => item.count),
                        backgroundColor: ['#8884d8', '#82ca9d', '#ffc658', '#ff7300', '#0088fe']
                    }]
                },
                options: {
                    responsive: true,
                    maintainAspectRatio: false
                }
            });

            new Chart(document.getElementById('topUrlsChart'), {
                type: 'bar',
                data: {
                    labels: stats.topUrls.map(item => item.url),
                    datasets: [{
                        label: 'Word Count',
                        data: stats.topUrls.map(item => item.count),
                        backgroundColor: '#ffc658'
                    }]
                },
                options: {
                    responsive: true,
                    maintainAspectRatio: false,
                    indexAxis: 'y'
                }
            });
        };

        ['search_input', 'nav_search_input'].forEach(id => {
            document.getElementById(id).addEventListener('keypress', function(event) {
                if (event.key === 'Enter') {
                    event.preventDefault();
                    performSearch(id === 'nav_search_input');
                }
            });
        });
    </script>
    """
    display(HTML(search_page_html))

def update_results(results_html):
    results_html = results_html.replace('`', '\\`').replace("'", "\\'").replace('\n', ' ')
    js_code = f"""
    const resultsElement = document.getElementById('searchResults');
    if (resultsElement) {{
        resultsElement.innerHTML = `{results_html}`;
    }} else {{
        console.error('Search results element not found');
    }}
    """
    display(Javascript(js_code))

def handle_search(args):
    try:
        search_terms = args[0]
        page = int(args[1]) if len(args) > 1 else 1
        results_per_page = 5

        query_service = DynamicQueryService(firebase_url)
        results = query_service.create_query(search_terms)

        total_results = len(results.get('results', []))
        total_pages = max(1, (total_results + results_per_page - 1) // results_per_page)
        page = min(max(1, page), total_pages)
        start_idx = (page - 1) * results_per_page
        end_idx = start_idx + results_per_page

        results_html = f"""
        <div style='max-width: 800px; margin: 0 auto; padding: 20px;'>
            <div style='margin-bottom: 20px; font-size: 16px; color:black'>
                Found {total_results} results for "{' '.join(search_terms)}"
            </div>
        """

        if 'error' in results:
            results_html += f"<div style='color: red; padding: 10px;'>Error: {results['error']}</div>"
        elif 'message' in results:
            results_html += f"<div style='padding: 10px;'>{results['message']}</div>"
        else:
            current_results = results['results'][start_idx:end_idx]
            for result in current_results:
                results_html += f"""
                    <div style='background: white;
                                padding: 20px;
                                margin-bottom: 15px;
                                border-radius: 10px;
                                box-shadow: 0 1px 3px rgba(0,0,0,0.1);'>
                        <div style='font-size: 18px; margin-bottom: 10px;'>
                            <a href='{result["url"]}'
                               target='_blank'
                               style='color: #1a0dab;
                                      text-decoration: none;'>
                                {result["url"].split("/")[-1].replace("-", " ").title()}
                            </a>
                        </div>
                        <div style='color: #006621;
                                    font-size: 14px;
                                    margin-bottom: 5px;'>
                            {result["url"]}
                        </div>
                        <div style='color: #545454;
                                    font-size: 13px;'>
                            Score: {result["score"]}
                        </div>
                    </div>
                """

            if total_pages > 1:
                results_html += """
                    <div style='text-align: center; margin-top: 30px;'>
                        <div style='display: inline-flex; align-items: center;
                                  background: white; padding: 10px; border-radius: 25px;
                                  box-shadow: 0 1px 3px rgba(0,0,0,0.1);'>
                """

                if page > 1:
                    results_html += f"""
                        <button onclick='performPageChange({page-1})'
                                style='background: none; border: none; cursor: pointer;
                                       color: #1a0dab; padding: 0 15px; font-size: 14px;'>
                            ← Previous
                        </button>
                    """

                for p in range(max(1, page-2), min(total_pages+1, page+3)):
                    if p == page:
                        results_html += f"""
                            <span style='color: white; background: #1a0dab;
                                       padding: 5px 12px; border-radius: 50%;
                                       margin: 0 5px; font-size: 14px;'>
                                {p}
                            </span>
                        """
                    else:
                        results_html += f"""
                            <button onclick='performPageChange({p})'
                                    style='background: none; border: none; cursor: pointer;
                                           color: #1a0dab; padding: 5px 12px; margin: 0 5px;
                                           font-size: 14px;'>
                                {p}
                            </button>
                        """

                if page < total_pages:
                    results_html += f"""
                        <button onclick='performPageChange({page+1})'
                                style='background: none; border: none; cursor: pointer;
                                       color: #1a0dab; padding: 0 15px; font-size: 14px;'>
                            Next →
                        </button>
                    """

                results_html += "</div></div>"

        results_html += "</div>"
        update_results(results_html)

    except Exception as e:
        import traceback
        print(f"Search error details: {traceback.format_exc()}")
        error_html = f"""
            <div style='color: red; padding: 20px;'>
                An error occurred while processing your search: {str(e)}
            </div>
        """
        update_results(error_html)

# Register the callback
output.register_callback('notebook.handleSearch', handle_search)

# Display the search page
display_search_page()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>